In [ ]:
import tvm
import numpy as np

import tvm.te as te
import tvm.relax as rx
import tvm.tir as tir

from tvm.script import ir as I
from tvm.script import tir as T
from tvm.script import relax as R

from tvm.relax import ExecBuilder


def showmod(mod: tvm.ir.module.IRModule):
    mod.show(
        black_format=True,
        show_meta=False,
        verbose_expr=True,
        show_object_address=False,
        show_all_struct_info=True,
    )


def createandshowmod(ops):
    te_func = te.create_prim_func(ops).with_attrs({"global_symbol": "test"})
    mod = tvm.IRModule({"test": te_func})
    showmod(mod)

In [ ]:
@tvm.script.ir_module
class Model:
    @R.function
    def main(x: R.Tensor([1, 3, 224, 224], "float32")):
        weight = R.const(np.random.randn(16, 3, 3, 3), "float32")
        conv = tvm.relax.op.nn.conv2d(x, weight)
        return conv

# Build the model
target = tvm.target.Target("llvm")
ex = tvm.relax.build(Model, target)

# Create vritual machine
dev = tvm.device(str(target), 0)
vm = tvm.relax.VirtualMachine(ex, dev)

# Prepare input data and transform to tvm.nd.array
input_data = np.random.randn(1, 3, 224, 224).astype("float32")
input_tvm = tvm.nd.array(input_data, dev)

# Execute
output = vm["main"](input_tvm)
print(output.numpy()[0][0][0][0:10])

[ 7.1757183   7.512619   -4.311302   -4.0354824  -0.50937843  3.5278237
 -4.9293637  -6.478323   -1.2307802   2.965952  ]


In [ ]:
eb = ExecBuilder()
eb.declare_function("test.vm.add", kind=tvm.relax.exec_builder.VMFuncKind.VM_FUNC)

# Register custom_add function
if not tvm.get_global_func("custom_add", allow_missing=True):
  @tvm.register_func("custom_add")
  def custom_add(a: tvm.nd.array, b: tvm.nd.array) -> tvm.nd.array:
      np_a = a.numpy()
      np_b = b.numpy()
      return tvm.nd.array(np_a + np_b)

with eb.function("test.vm.add", num_inputs=2):
    eb.emit_call("custom_add", args=[eb.r(0), eb.r(1)], dst=eb.r(2))
    eb.emit_ret(eb.r(2))

with eb.function("main", num_inputs=2):
    eb.emit_call("test.vm.add", args=[eb.r(0), eb.r(1)], dst=eb.r(2))
    eb.emit_ret(eb.r(2))

# Get execution module
ex = eb.get()

# Create virtual machine
dev = tvm.cpu()
vm = tvm.relax.VirtualMachine(ex, dev)

# Prepare input data and transform to tvm.nd.array
input1 = tvm.nd.array(np.array([1.0], dtype="float32"))
input2 = tvm.nd.array(np.array([2.0], dtype="float32"))

# Execute
result = vm["main"](input1, input2)
print(result.numpy())

[3.]
